In [1]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Define transformations for data preprocessing
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) # Normalize pixel values
])

# Download and load the CIFAR-10 training data
trainset = datasets.CIFAR10(root='./data', download=True, train=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=64, shuffle=True)


100.0%


Extracting ./data\cifar-10-python.tar.gz to ./data


In [2]:
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),   # Randomly flip images horizontally
    transforms.RandomCrop(32, padding=4),# Randomly crop with padding
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) # Normalize
])

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define a simple neural network
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(32*32*3, 256)
        self.fc2 = nn.Linear(256, 10)
        
    def forward(self, x):
        x = x.view(x.size(0), -1) # Flatten the image
        x = torch.relu(self.fc1(x))
        return self.fc2(x)

# Initialize the model, loss function, and optimizer
model = SimpleNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Training loop
for epoch in range(10): # 10 epochs
    running_loss = 0.0
    for images, labels in trainloader:
        optimizer.zero_grad()       # Zero the gradients
        
        outputs = model(images)     # Forward pass
        loss = criterion(outputs, labels) # Compute loss
        loss.backward()             # Backward pass
        optimizer.step()            # Update weights
        
        running_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(trainloader)}")


Epoch 1, Loss: 1.8709254017876238
Epoch 2, Loss: 1.6589904607409407
Epoch 3, Loss: 1.5755416613710507
Epoch 4, Loss: 1.5201085682415292
Epoch 5, Loss: 1.4757102959601165
Epoch 6, Loss: 1.4370714776656206
Epoch 7, Loss: 1.404117999784172
Epoch 8, Loss: 1.3726537450195273
Epoch 9, Loss: 1.3427332773080567
Epoch 10, Loss: 1.3163938802069106


In [4]:
# Validation data transformations (only normalization, no augmentation)
val_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize pixel values
])

# Download and load the CIFAR-10 validation data
valset = datasets.CIFAR10(root='./data', download=True, train=False, transform=val_transform)
valloader = DataLoader(valset, batch_size=64, shuffle=False)


Files already downloaded and verified


In [6]:
# Validation loop
val_loss = 0.0
correct = 0
total = 0

# Switch the model to evaluation mode
model.eval()

with torch.no_grad():  # No need to track gradients during validation
    for images, labels in valloader:
        outputs = model(images)
        loss = criterion(outputs, labels)  # Compute the loss on the validation set
        val_loss += loss.item()

        # Calculate accuracy
        _, predicted = torch.max(outputs, 1)  # Get the predicted class with the highest score
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# Calculate average loss and accuracy
avg_val_loss = val_loss / len(valloader)
accuracy = 100 * correct / total

print(f"Validation Loss: {avg_val_loss}, Validation Accuracy: {accuracy}%")


Validation Loss: 1.3849875372686204, Validation Accuracy: 51.47%


In [7]:
import torch.nn.functional as F

class SimpleNNWithDropout(nn.Module):
    def __init__(self):
        super(SimpleNNWithDropout, self).__init__()
        self.fc1 = nn.Linear(32*32*3, 256)
        self.dropout = nn.Dropout(0.5)  # Drop 50% of neurons
        self.fc2 = nn.Linear(256, 10)
        
    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)  # Apply dropout
        return self.fc2(x)


In [9]:
import torch
import torch.nn as nn
import torch.optim as optim



# Initialize the model, loss function, and optimizer
model = SimpleNNWithDropout()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Training loop
for epoch in range(10): # 10 epochs
    running_loss = 0.0
    for images, labels in trainloader:
        optimizer.zero_grad()       # Zero the gradients
        
        outputs = model(images)     # Forward pass
        loss = criterion(outputs, labels) # Compute loss
        loss.backward()             # Backward pass
        optimizer.step()            # Update weights
        
        running_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(trainloader)}")


Epoch 1, Loss: 1.9117810892327058
Epoch 2, Loss: 1.7335050897220212
Epoch 3, Loss: 1.6639243780499529
Epoch 4, Loss: 1.6227053821544208
Epoch 5, Loss: 1.5946909420935393
Epoch 6, Loss: 1.5666773291804907
Epoch 7, Loss: 1.540536953512665
Epoch 8, Loss: 1.52170181564053
Epoch 9, Loss: 1.5042652809406485
Epoch 10, Loss: 1.4876859393875923


In [10]:
# Validation loop
val_loss = 0.0
correct = 0
total = 0

# Switch the model to evaluation mode
model.eval()

with torch.no_grad():  # No need to track gradients during validation
    for images, labels in valloader:
        outputs = model(images)
        loss = criterion(outputs, labels)  # Compute the loss on the validation set
        val_loss += loss.item()

        # Calculate accuracy
        _, predicted = torch.max(outputs, 1)  # Get the predicted class with the highest score
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# Calculate average loss and accuracy
avg_val_loss = val_loss / len(valloader)
accuracy = 100 * correct / total

print(f"Validation Loss: {avg_val_loss}, Validation Accuracy: {accuracy}%")

Validation Loss: 1.4394868225049062, Validation Accuracy: 50.22%


In [11]:
class SimpleNNWithBatchNorm(nn.Module):
    def __init__(self):
        super(SimpleNNWithBatchNorm, self).__init__()
        self.fc1 = nn.Linear(32*32*3, 256)
        self.batch_norm = nn.BatchNorm1d(256)
        self.fc2 = nn.Linear(256, 10)
        
    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = F.relu(self.batch_norm(self.fc1(x)))  # Apply batch normalization
        return self.fc2(x)


In [12]:
import torch
import torch.nn as nn
import torch.optim as optim



# Initialize the model, loss function, and optimizer
model = SimpleNNWithBatchNorm()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Training loop
for epoch in range(10): # 10 epochs
    running_loss = 0.0
    for images, labels in trainloader:
        optimizer.zero_grad()       # Zero the gradients
        
        outputs = model(images)     # Forward pass
        loss = criterion(outputs, labels) # Compute loss
        loss.backward()             # Backward pass
        optimizer.step()            # Update weights
        
        running_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(trainloader)}")


Epoch 1, Loss: 1.7114399464234062
Epoch 2, Loss: 1.5290845512124278
Epoch 3, Loss: 1.450971159331329
Epoch 4, Loss: 1.3912510155411937
Epoch 5, Loss: 1.3434709112357606
Epoch 6, Loss: 1.3003548142855124
Epoch 7, Loss: 1.2631244843115892
Epoch 8, Loss: 1.2299953024558095
Epoch 9, Loss: 1.19897321971786
Epoch 10, Loss: 1.167670200471683


In [13]:
# Validation loop
val_loss = 0.0
correct = 0
total = 0

# Switch the model to evaluation mode
model.eval()

with torch.no_grad():  # No need to track gradients during validation
    for images, labels in valloader:
        outputs = model(images)
        loss = criterion(outputs, labels)  # Compute the loss on the validation set
        val_loss += loss.item()

        # Calculate accuracy
        _, predicted = torch.max(outputs, 1)  # Get the predicted class with the highest score
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# Calculate average loss and accuracy
avg_val_loss = val_loss / len(valloader)
accuracy = 100 * correct / total

print(f"Validation Loss: {avg_val_loss}, Validation Accuracy: {accuracy}%")

Validation Loss: 1.365519998939174, Validation Accuracy: 51.82%
